# LLM Sorting Demo (vLLM): "Can 5 Agents Sort Themselves?"

This notebook is designed for a lecture demo on sorting algorithms.  
Five LLM personas (with names already in alphabetical order) start in a shuffled line and **talk to each other** while performing adjacent comparisons.

## Learning goals
1. Visualize a sorting process step-by-step.
2. Show how local communication (neighbor-to-neighbor) can still produce a globally sorted order.
3. Count algorithmic cost and compare to theoretical optima.
4. Use `vllm` to generate live dialogue so students can *see and hear* what each comparison is doing.

## 1) Environment Setup

> This demo uses one base model through `vllm` and assigns five different personas.
> It works well with an instruction-tuned model (for example Llama-3-Instruct style checkpoints).

In [ ]:
# If needed, install dependencies in your environment:
# !pip install vllm matplotlib pandas ipywidgets

import math
import random
from dataclasses import dataclass
from typing import List, Dict, Any

import matplotlib.pyplot as plt
import matplotlib.patches as patches
import pandas as pd
from IPython.display import display, Markdown

from vllm import LLM, SamplingParams

In [ ]:
# ---- Configure your model path ----
# Examples:
# model_name = "meta-llama/Meta-Llama-3-8B-Instruct"
# model_name = "/models/Meta-Llama-3-8B-Instruct"
model_name = "meta-llama/Meta-Llama-3-8B-Instruct"

sampling_params = SamplingParams(
    temperature=0.6,
    top_p=0.9,
    max_tokens=120,
)

# Set True for a fast classroom dry-run without loading a model.
USE_MOCK_DIALOGUE = False

# Initialize once (can take time, depending on your hardware)
llm = None if USE_MOCK_DIALOGUE else LLM(model=model_name)

## Quick Preview Mode (before full vLLM run)

If you want to preview the demo instantly in class before loading a model:
1. Set `USE_MOCK_DIALOGUE = True`.
2. Run all cells.

You will still get the full sorting animation and step-count analysis, but dialogue is deterministic and fast.
Then switch back to `False` for live vLLM-generated agent conversations.

## 2) Define the Five LLM Personas

The names are intentionally alphabetical:

- **Aster**
- **Beryl**
- **Cygnus**
- **Dahlia**
- **Ember**

We then shuffle their starting order to create a sorting challenge.

In [ ]:
personas = ["Aster", "Beryl", "Cygnus", "Dahlia", "Ember"]

random.seed(140)
start_order = personas.copy()
random.shuffle(start_order)

start_order

In [ ]:
@dataclass
class StepRecord:
    step: int
    left_index: int
    right_index: int
    left_name: str
    right_name: str
    should_swap: bool
    order_after: List[str]
    left_message: str
    right_message: str


def inversion_count(arr: List[str]) -> int:
    """Number of adjacent swaps required by any adjacent-swap sorting process."""
    inv = 0
    for i in range(len(arr)):
        for j in range(i + 1, len(arr)):
            if arr[i] > arr[j]:
                inv += 1
    return inv


def info_theory_lower_bound_comparisons(n: int) -> int:
    """Lower bound on comparisons for any comparison sort: ceil(log2(n!))."""
    return math.ceil(math.log2(math.factorial(n)))

## 3) LLM Communication Function

For each adjacent pair, both agents speak:

- Left agent explains whether it should stay/swap.
- Right agent responds.

To keep the algorithm correct and stable for teaching, the **actual swap decision is made by exact lexical comparison** (`left_name > right_name`).
The model-generated text provides the social/visual explanation layer.

In [ ]:
def ask_agent(agent_name: str, other_name: str, side: str, should_swap: bool) -> str:
    decision_word = "SWAP" if should_swap else "KEEP"

    if USE_MOCK_DIALOGUE:
        verb = "move left" if should_swap else "hold position"
        return (
            f"I'm {agent_name} on the {side}; compared with {other_name}, I should {verb}. "
            f"Decision={decision_word}"
        )

    prompt = f"""
You are {agent_name}, an LLM persona standing next to {other_name} in a sorting line.
Your side: {side}.
Ground-truth decision from referee: {decision_word}.

In 1-2 short sentences:
1) Explain your local comparison using alphabetical order.
2) End with exactly: Decision={decision_word}
Tone: lively for a classroom demo.
""".strip()

    outputs = llm.generate([prompt], sampling_params)
    return outputs[0].outputs[0].text.strip()

In [ ]:
def draw_state(order: List[str], title: str, highlight: tuple = None):
    """
    Draw a horizontal lineup of personas.
    highlight: optional (i, j, swapped_bool)
    """
    n = len(order)
    fig, ax = plt.subplots(figsize=(12, 2.8))
    ax.set_xlim(0, n)
    ax.set_ylim(0, 1)
    ax.axis("off")

    for idx, name in enumerate(order):
        x = idx + 0.05
        w = 0.9
        h = 0.55
        color = "#7ec8e3"

        if highlight and idx in (highlight[0], highlight[1]):
            color = "#ffd166" if highlight[2] else "#caffbf"

        rect = patches.FancyBboxPatch(
            (x, 0.2), w, h,
            boxstyle="round,pad=0.02,rounding_size=0.04",
            linewidth=2,
            edgecolor="#1f2937",
            facecolor=color,
        )
        ax.add_patch(rect)
        ax.text(idx + 0.5, 0.48, name, ha="center", va="center", fontsize=12, weight="bold")

    if highlight:
        i, j, swapped = highlight
        action = "SWAP" if swapped else "KEEP"
        ax.annotate(
            action,
            xy=((i + j + 1)/2, 0.85),
            ha="center",
            fontsize=12,
            weight="bold",
            color="#111827",
        )

    plt.title(title, fontsize=14, weight="bold")
    plt.show()

## 4) Run the "Self-Sorting" Conversation (Odd-Even Neighbor Passes)

We use an adjacent-comparison strategy (bubble-style neighbor exchanges):

- Pass 1: compare (0,1), (1,2), (2,3), (3,4)
- Repeat passes until sorted.

This is perfect for a communication demo because each action is local and visible.

In [ ]:
def run_demo(initial_order: List[str]) -> Dict[str, Any]:
    order = initial_order.copy()
    history: List[StepRecord] = []
    step = 0

    draw_state(order, title="Initial lineup (shuffled)")

    n = len(order)
    for pass_num in range(1, n):  # bubble-sort style upper bound
        swapped_this_pass = False
        display(Markdown(f"### Pass {pass_num}"))

        for i in range(0, n - 1):
            j = i + 1
            left, right = order[i], order[j]
            should_swap = left > right

            left_msg = ask_agent(left, right, "LEFT", should_swap)
            right_msg = ask_agent(right, left, "RIGHT", should_swap)

            if should_swap:
                order[i], order[j] = order[j], order[i]
                swapped_this_pass = True

            step += 1
            rec = StepRecord(
                step=step,
                left_index=i,
                right_index=j,
                left_name=left,
                right_name=right,
                should_swap=should_swap,
                order_after=order.copy(),
                left_message=left_msg,
                right_message=right_msg,
            )
            history.append(rec)

            draw_state(
                order,
                title=f"Step {step}: compare {left} vs {right}",
                highlight=(i, j, should_swap),
            )

            display(Markdown("**{} says:** {}\n\n**{} says:** {}".format(left, left_msg, right, right_msg)))

        if not swapped_this_pass:
            break

    return {
        "final_order": order,
        "steps": step,
        "history": history,
    }

In [ ]:
results = run_demo(start_order)
results["final_order"], results["steps"]

## 5) Metrics: Steps vs Optimal

We compute:

1. **Our observed comparisons** = number of neighbor checks performed.
2. **Observed swaps** = number of inversions resolved during run.
3. **Optimal adjacent swaps** = inversion count of the initial permutation.
4. **Information-theoretic lower bound on comparisons** = `ceil(log2(5!)) = 7`.

For `n=5`, bubble-like neighbor strategies typically use more than 7 comparisons, which is a great teaching point: **local communication clarity vs comparison efficiency**.

In [ ]:
initial = start_order
final_order = results["final_order"]
history = results["history"]

observed_comparisons = len(history)
observed_swaps = sum(1 for h in history if h.should_swap)
optimal_adj_swaps = inversion_count(initial)
lb_comparisons = info_theory_lower_bound_comparisons(len(initial))

summary = pd.DataFrame([
    {"Metric": "Initial order", "Value": " -> ".join(initial)},
    {"Metric": "Final order", "Value": " -> ".join(final_order)},
    {"Metric": "Observed comparisons (this demo)", "Value": observed_comparisons},
    {"Metric": "Observed swaps (this demo)", "Value": observed_swaps},
    {"Metric": "Optimal adjacent swaps (= inversions)", "Value": optimal_adj_swaps},
    {"Metric": "Lower bound comparisons ceil(log2(5!))", "Value": lb_comparisons},
])
summary

In [ ]:
# Detailed step table for classroom walkthrough
rows = []
for h in history:
    rows.append({
        "Step": h.step,
        "Pair": f"({h.left_index},{h.right_index})",
        "Compared": f"{h.left_name} vs {h.right_name}",
        "Action": "SWAP" if h.should_swap else "KEEP",
        "Order after step": " -> ".join(h.order_after),
    })

pd.DataFrame(rows)

## 6) How to Explain This in Class

### Why this is "LLMs sorting themselves"
- Each comparison is narrated by both neighboring agents.
- The global ordering emerges from local pairwise interactions.
- Students can track **state transitions** visually after each step.

### Algorithmic angle
- With adjacent swaps, the exact minimum number of swaps equals the inversion count.
- But minimum number of comparisons for arbitrary comparison sorting is bounded by `log2(n!)`.
- This demo intentionally prioritizes interpretability and communication over optimal comparison count.

### Extensions
1. Replace bubble-style passes with insertion-sort dialogue.
2. Add a "moderator" agent that asks only one question per pair.
3. Let agents vote on uncertainty before referee enforces exact order.
4. Run multiple random initial orders and plot average steps.